## Import libraries

In [167]:
%matplotlib inline
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
from statsmodels.graphics.gofplots import qqplot
import seaborn as sns
from itertools import combinations
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.feature_selection import RFE
from numpy.polynomial.polynomial import polyfit
from pandas import Series
import matplotlib.pylab as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
import datetime
from dateutil.relativedelta import relativedelta
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.graphics.tsaplots import plot_acf
from matplotlib.pylab import rcParams
from statsmodels.tsa.arima_model import ARMA

## Load data

In [60]:
!pwd

/Users/flatironschool/Documents/Mod_4_Project_Folder/mod_4_project


In [67]:
!ls

README.md                 combine_data.csv          mod_4_proj_notebook.ipynb


In [68]:
# https://www.kaggle.com/savvastj/nfl-combine-data

In [135]:
combine_data = pd.read_csv('combine_data.csv')

## Get data ready

In [136]:
combine_data.Pick.fillna(0,inplace=True)
combine_data = combine_data[combine_data.Pick>0]
combine_data.set_index('Player',inplace=True)

In [137]:
#combine_data.drop(['Pfr_ID','AV','Team','Round','Year'],axis=1,inplace=True)

In [138]:
combine_data.Forty.fillna(0,inplace=True)
combine_data = combine_data[combine_data.Forty>0]
combine_data.Vertical.fillna(0,inplace=True)
combine_data = combine_data[combine_data.Vertical>0]
combine_data.BenchReps.fillna(0,inplace=True)
combine_data = combine_data[combine_data.BenchReps>0]
combine_data.BroadJump.fillna(0,inplace=True)
combine_data = combine_data[combine_data.BroadJump>0]
combine_data.Cone.fillna(0,inplace=True)
combine_data = combine_data[combine_data.Cone>0]
combine_data.Shuttle.fillna(0,inplace=True)
combine_data = combine_data[combine_data.Shuttle>0]

In [184]:
print(combine_data.shape)
combine_data.head()

(1822, 10)


,Pos,Ht,Wt,Forty,Vertical,BenchReps,BroadJump,Cone,Shuttle,Pick
Player,,,,,,,,,,
Darnell Alford,OT,76,334,5.56,25.0,23.0,94.0,8.48,4.98,188.0
Reggie Austin,CB,69,175,4.44,35.0,17.0,119.0,7.03,4.14,125.0
Rashidi Barnes,FS,72,208,4.62,35.0,10.0,114.0,6.92,4.32,225.0
David Barrett,CB,70,199,4.44,37.5,16.0,116.0,6.81,4.04,102.0
Rogers Beckett,FS,73,207,4.62,39.5,15.0,119.0,6.48,4.29,43.0


## Categorical features

In [177]:
combine_data_cat = pd.get_dummies(combine_data.Pos)

In [178]:
combine_data_cat.drop('C',axis=1,inplace=True)

In [179]:
combine_data_cat.head(1)

,CB,DE,DT,FB,FS,ILB,LS,OG,OLB,OT,QB,RB,SS,TE,WR
Player,,,,,,,,,,,,,,,
Darnell Alford,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [180]:
combine_atg = combine_data.copy()
combine_atg.drop('Pos',axis=1,inplace=True)
combine_atg['CB']=combine_data_cat['CB']
combine_atg['DE']=combine_data_cat['DE']
combine_atg['DT']=combine_data_cat['DT']
combine_atg['FB']=combine_data_cat['FB']
combine_atg['FS']=combine_data_cat['FS']
combine_atg['ILB']=combine_data_cat['ILB']
combine_atg['LS']=combine_data_cat['LS']
combine_atg['OG']=combine_data_cat['OG']
combine_atg['OT']=combine_data_cat['OT']
combine_atg['QB']=combine_data_cat['QB']
combine_atg['OLB']=combine_data_cat['OLB']
combine_atg['SS']=combine_data_cat['SS']
combine_atg['TE']=combine_data_cat['TE']
combine_atg['WR']=combine_data_cat['WR']

In [185]:
combine_atg.drop('Pick',axis=1,inplace=True)

In [186]:
print(combine_atg.shape)
combine_atg.head()

(1822, 22)


,Ht,Wt,Forty,Vertical,BenchReps,BroadJump,Cone,Shuttle,CB,DE,...,FS,ILB,LS,OG,OT,QB,OLB,SS,TE,WR
Player,,,,,,,,,,,,,,,,,,,,,
Darnell Alford,76,334,5.56,25.0,23.0,94.0,8.48,4.98,0,0,...,0,0,0,0,1,0,0,0,0,0
Reggie Austin,69,175,4.44,35.0,17.0,119.0,7.03,4.14,1,0,...,0,0,0,0,0,0,0,0,0,0
Rashidi Barnes,72,208,4.62,35.0,10.0,114.0,6.92,4.32,0,0,...,1,0,0,0,0,0,0,0,0,0
David Barrett,70,199,4.44,37.5,16.0,116.0,6.81,4.04,1,0,...,0,0,0,0,0,0,0,0,0,0
Rogers Beckett,73,207,4.62,39.5,15.0,119.0,6.48,4.29,0,0,...,1,0,0,0,0,0,0,0,0,0


## Train - Test - Split

In [ ]:
df_features = combine_data[['Pos','Ht','Wt','Forty','Vertical','BenchReps','BroadJump','Cone','Shuttle']]

In [ ]:
df_target = combine_data[['Pick']]

In [156]:
X_train, X_test, y_train, y_test = train_test_split(df_features,df_target,test_size=0.25,random_state=45)

In [162]:
print(X_train.shape[0]==y_train.shape[0])
print(X_test.shape[0]==y_test.shape[0])

True
True


## Check messy model

In [168]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()
linreg.fit(df_features, df_target)

ValueError: could not convert string to float: 'OT'

In [166]:
yhat = linreg.predict(df_features)
SS_Residual = np.sum((df_target-yhat)**2)
SS_Total = np.sum((df_target-np.mean(df_target))**2)
r_squared = 1 - (float(SS_Residual))/SS_Total
adjusted_r_squared = 1 - (1-r_squared)*(len(df_target)-1)/(len(df_target)-df_features.shape[1]-1)

NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [169]:
mod = sm.OLS(df_target,df_features).fit()
mod.summary()

ValueError: Pandas data cast to numpy dtype of object. Check input data with np.asarray(data).